In [2]:
import pandas as pd
import numpy as np
import nltk
import random
from nltk.corpus import stopwords
from nltk.classify import NaiveBayesClassifier  
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import naive_bayes
from sklearn import svm
import os

In [3]:
# Read in the data
path="C:/Users/dawit/Google Drive/Data Mining/project"
filename_read = os.path.join(path,"text_emotion.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])
df


,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ..."
7,1956968636,worry,mcsleazy,Hmmm. http://www.djhero.com/ is down
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?


In [4]:
df=df.drop('tweet_id', axis= 1)
df=df.drop('author', axis= 1)
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [5]:
df['Emotion level'] = np.where(df['sentiment'].str.contains('sadness'), 1,np.where(df['sentiment'].str.contains('worry'), 2,np.where(df['sentiment'].str.contains('anger'), 3,np.where(df['sentiment'].str.contains('hate'), 4,np.where(df['sentiment'].str.contains('boredom'), 5,np.where(df['sentiment'].str.contains('enthusiasm'), 10,np.where(df['sentiment'].str.contains('neutral'), 6,np.where(df['sentiment'].str.contains('fun'), 8,np.where(df['sentiment'].str.contains('happiness'), 9,np.where(df['sentiment'].str.contains('relief'), 7,np.where(df['sentiment'].str.contains('surprise'), 11,np.where(df['sentiment'].str.contains('love'), 12,0))))))))))))
df.head()

,sentiment,content,Emotion level
0,empty,@tiffanylue i know i was listenin to bad habi...,0
1,sadness,Layin n bed with a headache ughhhh...waitin o...,1
2,sadness,Funeral ceremony...gloomy friday...,1
3,enthusiasm,wants to hang out with friends SOON!,10
4,neutral,@dannycastillo We want to trade with someone w...,6


In [19]:
X_train, X_test, y_train, y_test = train_test_split(df['content'], 
                                                    df['Emotion level'],test_size = 0.33,
                                                    random_state=0)

In [20]:
# Fit the CountVectorizer to the training data
vect = CountVectorizer().fit(X_train)
vect.get_feature_names()[::2000]
len(vect.get_feature_names())

36449

In [21]:
vect.get_feature_names()[::2000]

['00',
 'alex_navarro',
 'beats',
 'carleigh',
 'cravings',
 'dpocza',
 'fishhhface',
 'hairdressers',
 'irishpixie36',
 'kirstie',
 'madnessofmany',
 'mrmiistro',
 'overgrown',
 'qcy8d4',
 'saralds',
 'soundtracks',
 'theamazon111',
 'ustre',
 'yellowish']

In [22]:
len(vect.get_feature_names())

36449

In [23]:
X_train_vectorized = vect.transform(X_train)
X_train_vectorized

<26800x36449 sparse matrix of type '<class 'numpy.int64'>'
	with 317981 stored elements in Compressed Sparse Row format>

In [24]:
# Train the model
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)
# Predict the transformed test documents
predictions = model.predict(vect.transform(X_test))
print('Accurarcy: ',accuracy_score(y_test, predictions))

Accurarcy:  0.342424242424


In [20]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
def plot_confusion_matrix(cm, names, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(names))
    plt.xticks(tick_marks, names, rotation=45)
    plt.yticks(tick_marks, names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
cm = confusion_matrix(y_test, predictions)
np.set_printoptions(precision=2)
print('Confusion matrix')
print(cm)
#plt.figure()
#plot_confusion_matrix(cm, X_test)
#plt.show()


Confusion matrix
[[   0    0    1    0    0    7    1    1   10    1    3    1   12]
 [   0    0    0    0    2    3    3    1   17    0   11    0   23]
 [   0    1    1    1    1   14    2    4  145    2   18    3   64]
 [   0    1    1    1    5   45    3   13   93    1   17    5   54]
 [   0    0    0    4   29  128    4   36  202    7   41   13  103]
 [   0    0    2    3   43  586    3  168  520   26   59   44  262]
 [   0    2    0    0    5   18   66    4   97    0   80    8  125]
 [   0    0    0    0   17  259    8  497  269   17   63   16  153]
 [   1    1    6    5   32  227   22  105 1630   20  198   42  582]
 [   0    0    1    0   12   85    2   29  177   24   27    7  121]
 [   0    0    4    1   11   79   48   52  433    9  457   15  632]
 [   0    0    0    1    8  101    9   41  270    6   54   32  198]
 [   0    0    3    1   19  153   51   71  828   24  413   44 1197]]


In [13]:
clfrNB = naive_bayes.MultinomialNB()
clfrNB.fit(X_train_vectorized,y_train)
X_test_vectorized = vect.transform(X_test)
predicted_labels = clfrNB.predict(X_test_vectorized)
from sklearn import metrics
metrics.f1_score(y_test,predicted_labels,average='micro')

0.30363636363636365

0.30363636363636365

In [14]:
print('Accurarcy: ',accuracy_score(y_test, predicted_labels))

Accurarcy:  0.303636363636


In [15]:
from sklearn import svm
clfrSVM = svm.SVC(kernel='linear', C=0.1)
clfrSVM.fit(X_train_vectorized, y_train)
predicted_labels = clfrSVM.predict(X_test_vectorized)
metrics.f1_score(y_test,predicted_labels,average='micro')

0.35325757575757577

0.35325757575757577

In [37]:
predicted_labels = model_selection.cross_val_predict(clfrSVM,
X_train_vectorized, y_train, cv=5)

In [38]:
metrics.f1_score(y_test,predicted_labels,average='micro')

ValueError: Found input variables with inconsistent numbers of samples: [13200, 26800]

In [25]:
# get the feature names as numpy array
feature_names = np.array(vect.get_feature_names())

# Sort the coefficients from the model
sorted_coef_index = model.coef_[0].argsort()


print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['love' 'sick' 'best' 'happy' 'fun' 'actually' 'sad' 'wow' 'amazing' 'wish']

Largest Coefs: 
['donniewahlberg' 'lights' 'weee' 'bored' 'tip' 'exactly' 'stinks' 'sheep'
 'complain' 'katy']


In [27]:
# Fit the TfidfVectorizer to the training data specifiying a minimum document frequency of 5
vect = TfidfVectorizer(min_df=10).fit(X_train)
len(vect.get_feature_names())
X_train_vectorized = vect.transform(X_train)

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))
print('Accurarcy: ',accuracy_score(y_test, predictions))

Accurarcy:  0.351742424242


In [50]:
clfrNB = naive_bayes.MultinomialNB()
clfrNB.fit(X_train_vectorized,y_train)
X_test_vectorized = vect.transform(X_test)
predicted_labels = clfrSVM.predict(X_test_vectorized)
metrics.f1_score(y_test,predicted_labels,average='micro')

0.3175

In [49]:
clfrSVM = svm.SVC(kernel='linear', C=0.1)
clfrSVM.fit(X_train_vectorized, y_train)
predicted_labels = clfrSVM.predict(X_test_vectorized)
metrics.f1_score(y_test,predicted_labels,average='micro')

0.3175

In [31]:
# document frequency of 5 and extracting 1-grams and 2-grams
vect = CountVectorizer(min_df=5, ngram_range=(1,10)).fit(X_train)

X_train_vectorized = vect.transform(X_train)

len(vect.get_feature_names())

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))
print('Accurarcy: ',accuracy_score(y_test, predictions))


Accurarcy:  0.325


In [8]:
clfrSVM = svm.SVC(kernel='linear', C=0.1)
clfrSVM.fit(X_train_vectorized, y_train)
predicted_labels = clfrSVM.predict(X_test_vectorized)
metrics.f1_score(y_test,predicted_labels,average='micro')

NameError: name 'X_train_vectorized' is not defined

In [ ]:
vect = CountVectorizer(min_df=5, ngram_range=(1,2)).fit(X_train)

X_train_vectorized = vect.transform(X_train)

len(vect.get_feature_names())